In [23]:
from ngsolve import *
from ngsolve.webgui import Draw
import random
import numpy as np

#Defining parameters
epsilon = 0.02
gamma = 50
M = 1e-3
maxh = 0.01 
order = 1 
tau = 0.1            
time = 0.0
T = 1


#Defining mesh and Finite element space
mesh = Mesh(unit_square.GenerateMesh(maxh=maxh))
V = H1(mesh, order=1) 

fes = V*V #FESpace([V,V])

phi1,mu1 = fes.TrialFunction()  #n+1
v,w = fes.TestFunction()

phi_old = GridFunction(V)
#Initialising phi_0 random values between -1 and 1
random_values = np.random.uniform(-1, 1, V.ndof)
#phi_old.vec.FV().NumPy()[:] = random_values
phi_old.vec.data = random_values

#f=LinearForm(fes)
#f += tau*rhs_1*v*dx+gamma*(phi_old**3-phi_old)*w*dx+ phi_old*v*dx

A = BilinearForm(fes)
A += phi1*v*dx+tau*M*grad(mu1)*grad(v)*dx + mu1*w*dx-epsilon*grad(phi1)*grad(w)*dx
A.Assemble()

t = 0

gfu = GridFunction(fes)

gfut = GridFunction(gfu.space, multidim = 0)
gfut.AddMultiDimComponent(gfu.vec)


while t<T:
    t+=tau
    rhs = exp(-t)*x
    
    f=LinearForm(fes)
    f+= tau*rhs*v*dx+gamma*(phi_old**3-phi_old)*w*dx+ phi_old*v*dx
    f.Assemble()

    gfu.vec.data = A.mat.Inverse(freedofs = fes.FreeDofs())*f.vec

    gfut.AddMultiDimComponent(gfu.vec)
    phi_old.vec.data = gfu.components[0].vec

    
    



In [24]:
Draw(gfut.components[0], mesh, interpolate_multidim = True, animate = True)
#Draw(gfu.components[0])

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

## Fully explicit scheme

Discretize $\partial_t \phi = \frac{\phi^{n+1}-\phi^n}{\tau}$, $\mu = \mu^{n+1}$, $\phi = \phi^n$ and $\nabla \phi = \nabla \phi^{n+1}$.

$$
\begin{aligned}
(\frac{\phi^{n+1}-\phi^n}{\tau},v) + (M\nabla \mu^{n+1}, \nabla v) &= 0\ \ \ \ \forall v \in \mathbb{V} \\
(\mu^{n+1},w)-(\epsilon \nabla \phi^{n+1}, \nabla w) 
&= (\gamma ((\phi^n)^3 -\phi^n),w) \ \ \ \  \forall w \in \mathbb{V} 
\end{aligned}
$$


## IMEX scheme


Discretize $\partial_t \phi = \frac{\phi^{n+1}-\phi^n}{\tau}$, $\mu = \mu^{n+1}$, $\phi = \phi^n$ and $\nabla \phi = \nabla \phi^{n+1}$.

$$
\begin{aligned}
(\frac{\phi^{n+1}-\phi^n}{\tau},v) + (M\nabla \mu^{n+1}, \nabla v) &= 0\ \ \ \ \forall v \in \mathbb{V} \\
(\mu^{n+1},w)-(\gamma[(\phi^n)^2-1]\phi^{n+1},w)-(\epsilon \nabla \phi^{n+1}, \nabla w) 
&= 0 \ \ \ \  \forall w \in \mathbb{V} 
\end{aligned}
$$

In [25]:
from ngsolve import *
from ngsolve.webgui import Draw
import random
import numpy as np
# Defining parameters
epsilon = 0.02
gamma = 50
M = 1e-3
maxh = 0.01 
order = 1
tau = 0.1            
T = 1
#Defining mesh and Finite element space
mesh = Mesh(unit_square.GenerateMesh(maxh=maxh))
V = H1(mesh, order=1) 
fes = V*V 

#Defining trial and test functions
phi,mu = fes.TrialFunction()  #n+1
v,w = fes.TestFunction()

#Initialising phi_0 random values between -1 and 1
phi_old = GridFunction(V)
#random_values = np.random.uniform(-1, 1, V.ndof)
random_values= np.zeros(V.ndof)
phi_old.vec.data = random_values

Kilde:https://docu.ngsolve.org/nightly/i-tutorials/unit-3.1-parabolic/parabolic.html

https://docu.ngsolve.org/latest/whetting_the_appetite/navierstokes.html

In [26]:
def phi_exact(t):
    return t*cos(np.pi*x)*cos(np.pi*y)

def phi_dt(t):
    return cos(np.pi*x)*cos(np.pi*y)

def phi_lap(t):
    return -2*np.pi**2*phi_exact(t)

def mu_exact():
    return cos(np.pi*x)*cos(np.pi*y)

def mu_lap():
    return -2*np.pi**2*mu_exact()

In [27]:
def IMEXScheme(mesh,fes,phi,mu,v,w,phi_old,epsilon,gamma,M,tau,T):
    
    gfu = GridFunction(fes)
    gfut = GridFunction(gfu.space, multidim = 0)
    gfut.AddMultiDimComponent(gfu.vec)
    t=0
    L2_error = []
    A = BilinearForm(fes)
    A += phi*v*dx+tau*M*grad(mu)*grad(v)*dx -gamma*(phi_exact(t)**2-1)*phi*w*dx+ mu*w*dx-epsilon*grad(phi)*grad(w)*dx

    #t=0
    while t<T:
        t+=tau

         #Defining Bilinear form
        #A = BilinearForm(fes)
        #A += phi*v*dx+tau*M*grad(mu)*grad(v)*dx -gamma*(phi_exact(t)**2-1)*phi*w*dx+ mu*w*dx-epsilon*grad(phi)*grad(w)*dx
        A.Assemble()

        rhs1 = phi_dt(t)-M*mu_lap() 
        rhs2 = mu_exact() - gamma*(phi_exact(t)**3 - phi_exact(t)) + epsilon*phi_lap(t)
            
        #Defining linear form
        f=LinearForm(fes)
        f+= tau*rhs1*v*dx+ phi_old*v*dx + rhs2*w*dx
        f.Assemble()

        #Solving for phi
        gfu.vec.data = A.mat.Inverse(freedofs = fes.FreeDofs())*f.vec

        e_h = gfu.components[0]-phi_exact(t)
        norm = np.sqrt(Integrate(e_h*e_h,mesh))
        L2_error.append(norm)

        #Storing and updating phi
        gfut.AddMultiDimComponent(gfu.vec)
        phi_old.vec.data = gfu.components[0].vec
        #t += tau

    return gfut,gfu, L2_error

In [28]:
M = 10**(-3)
gfut,gfu, L2_error = IMEXScheme(mesh,fes,phi,mu,v,w,phi_old,epsilon,gamma,M,tau,T)
Draw(gfut.components[0], mesh, interpolate_multidim = True, animate = True)

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene